In [9]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line.lower()) for line in lines]
lines = [" ".join(re.findall(r"\w+",line.lower())) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line.lower()) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line.lower())) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [2]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [3]:
print(pairs[:5])
print(input_docs[:5])

[('hola', 'buenos días en que puedo servirle'), ('tienes domicilio', 'claro que si que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'la arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'claro el chocolate un costo de 1000 sin leche'), ('y con leche', 'con lehe tiene un costo de 2000 pesos')]
['hola', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'y con leche']


In [4]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('model/training_model.h5')

Epoch 1/600
21/21 [==============================] - 8s 146ms/step - loss: 1.3733 - accuracy: 0.0253 - val_loss: 1.2867 - val_accuracy: 0.0331
Epoch 2/600
21/21 [==============================] - 1s 55ms/step - loss: 1.3091 - accuracy: 0.0344 - val_loss: 1.2758 - val_accuracy: 0.0337
Epoch 3/600
21/21 [==============================] - 1s 55ms/step - loss: 1.2177 - accuracy: 0.0346 - val_loss: 1.2793 - val_accuracy: 0.0316
Epoch 4/600
21/21 [==============================] - 1s 55ms/step - loss: 1.2592 - accuracy: 0.0336 - val_loss: 1.2799 - val_accuracy: 0.0342
Epoch 5/600
21/21 [==============================] - 1s 55ms/step - loss: 1.2099 - accuracy: 0.0358 - val_loss: 1.2838 - val_accuracy: 0.0337
Epoch 6/600
21/21 [==============================] - 1s 56ms/step - loss: 1.1410 - accuracy: 0.0382 - val_loss: 1.2866 - val_accuracy: 0.0347
Epoch 7/600
21/21 [==============================] - 1s 55ms/step - loss: 1.1453 - accuracy: 0.0386 - val_loss: 1.2894 - val_accuracy: 0.0357
Epoch

21/21 [==============================] - 1s 55ms/step - loss: 0.8952 - accuracy: 0.0967 - val_loss: 1.2879 - val_accuracy: 0.0648
Epoch 59/600
21/21 [==============================] - 1s 56ms/step - loss: 0.8417 - accuracy: 0.1048 - val_loss: 1.2643 - val_accuracy: 0.0663
Epoch 60/600
21/21 [==============================] - 1s 55ms/step - loss: 0.9067 - accuracy: 0.0970 - val_loss: 1.2561 - val_accuracy: 0.0699
Epoch 61/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8221 - accuracy: 0.1032 - val_loss: 1.2436 - val_accuracy: 0.0653
Epoch 62/600
21/21 [==============================] - 1s 57ms/step - loss: 0.8529 - accuracy: 0.1043 - val_loss: 1.2618 - val_accuracy: 0.0694
Epoch 63/600
21/21 [==============================] - 1s 60ms/step - loss: 0.8523 - accuracy: 0.1033 - val_loss: 1.2642 - val_accuracy: 0.0724
Epoch 64/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8010 - accuracy: 0.1116 - val_loss: 1.2675 - val_accuracy: 0.0678
Epoch 65/600

Epoch 115/600
21/21 [==============================] - 1s 57ms/step - loss: 0.5775 - accuracy: 0.1662 - val_loss: 1.2318 - val_accuracy: 0.0831
Epoch 116/600
21/21 [==============================] - 1s 57ms/step - loss: 0.6099 - accuracy: 0.1720 - val_loss: 1.2096 - val_accuracy: 0.0913
Epoch 117/600
21/21 [==============================] - 1s 57ms/step - loss: 0.5374 - accuracy: 0.1787 - val_loss: 1.2944 - val_accuracy: 0.0745
Epoch 118/600
21/21 [==============================] - 1s 56ms/step - loss: 0.6337 - accuracy: 0.1675 - val_loss: 1.2273 - val_accuracy: 0.0831
Epoch 119/600
21/21 [==============================] - 1s 58ms/step - loss: 0.5343 - accuracy: 0.1692 - val_loss: 1.2268 - val_accuracy: 0.0847
Epoch 120/600
21/21 [==============================] - 1s 58ms/step - loss: 0.5284 - accuracy: 0.1751 - val_loss: 1.2284 - val_accuracy: 0.0867
Epoch 121/600
21/21 [==============================] - 1s 58ms/step - loss: 0.5121 - accuracy: 0.1769 - val_loss: 1.2092 - val_accuracy:

Epoch 172/600
21/21 [==============================] - 1s 56ms/step - loss: 0.4151 - accuracy: 0.1960 - val_loss: 1.2403 - val_accuracy: 0.0841
Epoch 173/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3677 - accuracy: 0.1888 - val_loss: 1.2283 - val_accuracy: 0.0887
Epoch 174/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3640 - accuracy: 0.1883 - val_loss: 1.2264 - val_accuracy: 0.0847
Epoch 175/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3906 - accuracy: 0.1992 - val_loss: 1.2302 - val_accuracy: 0.0831
Epoch 176/600
21/21 [==============================] - 1s 56ms/step - loss: 0.4431 - accuracy: 0.2047 - val_loss: 1.2303 - val_accuracy: 0.0816
Epoch 177/600
21/21 [==============================] - 1s 57ms/step - loss: 0.3855 - accuracy: 0.2076 - val_loss: 1.2426 - val_accuracy: 0.0816
Epoch 178/600
21/21 [==============================] - 1s 58ms/step - loss: 0.3935 - accuracy: 0.2038 - val_loss: 1.2267 - val_accuracy:

Epoch 229/600
21/21 [==============================] - 2s 77ms/step - loss: 0.3333 - accuracy: 0.2227 - val_loss: 1.2241 - val_accuracy: 0.0862
Epoch 230/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3367 - accuracy: 0.2146 - val_loss: 1.2417 - val_accuracy: 0.0852
Epoch 231/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3128 - accuracy: 0.2134 - val_loss: 1.2401 - val_accuracy: 0.0887
Epoch 232/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3409 - accuracy: 0.2197 - val_loss: 1.2404 - val_accuracy: 0.0841
Epoch 233/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3270 - accuracy: 0.2131 - val_loss: 1.2446 - val_accuracy: 0.0831
Epoch 234/600
21/21 [==============================] - 1s 57ms/step - loss: 0.3445 - accuracy: 0.2255 - val_loss: 1.2331 - val_accuracy: 0.0877
Epoch 235/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3562 - accuracy: 0.2152 - val_loss: 1.2470 - val_accuracy:

...

21/21 [==============================] - 1s 57ms/step - loss: 0.2501 - accuracy: 0.2245 - val_loss: 1.2924 - val_accuracy: 0.0811
Epoch 343/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2539 - accuracy: 0.2360 - val_loss: 1.2897 - val_accuracy: 0.0836
Epoch 344/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2908 - accuracy: 0.2336 - val_loss: 1.2805 - val_accuracy: 0.0847
Epoch 345/600
21/21 [==============================] - 1s 55ms/step - loss: 0.2493 - accuracy: 0.2381 - val_loss: 1.2698 - val_accuracy: 0.0877
Epoch 346/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2329 - accuracy: 0.2281 - val_loss: 1.2803 - val_accuracy: 0.0877
Epoch 347/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2383 - accuracy: 0.2301 - val_loss: 1.2652 - val_accuracy: 0.0892
Epoch 348/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2537 - accuracy: 0.2275 - val_loss: 1.2789 - val_accuracy: 0.0831
Epoch 

Epoch 399/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2167 - accuracy: 0.2412 - val_loss: 1.2730 - val_accuracy: 0.0841
Epoch 400/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2432 - accuracy: 0.2355 - val_loss: 1.2996 - val_accuracy: 0.0821
Epoch 401/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2297 - accuracy: 0.2306 - val_loss: 1.3011 - val_accuracy: 0.0836
Epoch 402/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2347 - accuracy: 0.2366 - val_loss: 1.2801 - val_accuracy: 0.0872
Epoch 403/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2501 - accuracy: 0.2410 - val_loss: 1.3385 - val_accuracy: 0.0801
Epoch 404/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2055 - accuracy: 0.2351 - val_loss: 1.3321 - val_accuracy: 0.0796
Epoch 405/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2123 - accuracy: 0.2382 - val_loss: 1.3184 - val_accuracy:

Epoch 456/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1847 - accuracy: 0.2393 - val_loss: 1.3000 - val_accuracy: 0.0826
Epoch 457/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2084 - accuracy: 0.2466 - val_loss: 1.3142 - val_accuracy: 0.0790
Epoch 458/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2131 - accuracy: 0.2373 - val_loss: 1.3176 - val_accuracy: 0.0821
Epoch 459/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2138 - accuracy: 0.2413 - val_loss: 1.3203 - val_accuracy: 0.0811
Epoch 460/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2132 - accuracy: 0.2481 - val_loss: 1.3245 - val_accuracy: 0.0790
Epoch 461/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2191 - accuracy: 0.2391 - val_loss: 1.3084 - val_accuracy: 0.0836
Epoch 462/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1930 - accuracy: 0.2356 - val_loss: 1.3081 - val_accuracy:

Epoch 513/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1994 - accuracy: 0.2559 - val_loss: 1.3348 - val_accuracy: 0.0765
Epoch 514/600
21/21 [==============================] - 1s 55ms/step - loss: 0.1702 - accuracy: 0.2370 - val_loss: 1.3280 - val_accuracy: 0.0796
Epoch 515/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1878 - accuracy: 0.2402 - val_loss: 1.3575 - val_accuracy: 0.0724
Epoch 516/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1739 - accuracy: 0.2341 - val_loss: 1.3450 - val_accuracy: 0.0770
Epoch 517/600
21/21 [==============================] - 1s 55ms/step - loss: 0.1913 - accuracy: 0.2449 - val_loss: 1.3407 - val_accuracy: 0.0750
Epoch 518/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1979 - accuracy: 0.2531 - val_loss: 1.3466 - val_accuracy: 0.0750
Epoch 519/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1787 - accuracy: 0.2384 - val_loss: 1.3416 - val_accuracy:

Epoch 570/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1593 - accuracy: 0.2478 - val_loss: 1.3682 - val_accuracy: 0.0745
Epoch 571/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1476 - accuracy: 0.2395 - val_loss: 1.3479 - val_accuracy: 0.0821
Epoch 572/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1643 - accuracy: 0.2471 - val_loss: 1.3729 - val_accuracy: 0.0714
Epoch 573/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1494 - accuracy: 0.2409 - val_loss: 1.3229 - val_accuracy: 0.0801
Epoch 574/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1666 - accuracy: 0.2471 - val_loss: 1.3450 - val_accuracy: 0.0816
Epoch 575/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1453 - accuracy: 0.2394 - val_loss: 1.3438 - val_accuracy: 0.0785
Epoch 576/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1512 - accuracy: 0.2481 - val_loss: 1.3439 - val_accuracy:

In [7]:
from keras.models import load_model
training_model = load_model('model/training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [8]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Hola
 nuestro menu el dia de hoy es almuerzos
Buenos días
 el días de reservas son de lunes 
Tienes pizza?
 con lehe tiene un costo de 2000 pesos
chao
Que tengas un hermoso día!


In [ ]:
chatbot.start_chat()